# 1. Initials imports and sessions

In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/combinations1", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/teams", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/ro16", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/pgs_matches", True)

Out[42]: False

In [0]:
%python
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, rand, when
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
import random

In [0]:
%python
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("EuroCup") \
    .getOrCreate()

# 2. First table: Teams according to EURO 2024


In [0]:
%python
teams_data = [
    # Group A
    (1, "Germany", "A"),
    (2, "Hungary", "A"),
    (3, "Scotland", "A"),
    (4, "Switzerland", "A"),

    # Group B
    (5, "Albania", "B"),
    (6, "Croatia", "B"),
    (7, "Italy", "B"),
    (8, "Spain", "B"),

    # Group C
    (9, "Denmark", "C"),
    (10, "England", "C"),
    (11, "Slovenia", "C"),
    (12, "Serbia", "C"),

    # Group D
    (13, "Austria", "D"),
    (14, "France", "D"),
    (15, "Netherlands", "D"),
    (16, "Playoff Winner 1", "D"),

    # Group E
    (17, "Belgium", "E"),
    (18, "Romania", "E"),
    (19, "Slovakia", "E"),
    (20, "Playoff Winner 2", "E"),

    # Group F
    (21, "Czech Republic", "F"),
    (22, "Portugal", "F"),
    (23, "Turkey", "F"),
    (24, "Playoff Winner 3", "F")
]

In [0]:
%python
teams_columns = ["TeamID", "TeamName", "TeamGroup"]
teams_df = spark.createDataFrame(teams_data, teams_columns)
teams_df.write.saveAsTable("teams")

# spark.sql("SELECT * FROM teams").show()

# 3. Preparing for GroupStages

In [0]:
%python
matches_df = spark.sql("""
    SELECT  t1.TeamName AS HomeTeam, t2.TeamName AS AwayTeam, t1.TeamGroup AS HomeTeamGroup, t2.TeamGroup AS AwayTeamGroup
    FROM teams t1
    JOIN teams t2 ON t1.TeamGroup = t2.TeamGroup AND t1.TeamID < t2.TeamID  /*(TeamA, TeamB) != (TeamB, TeamA)*/
""")

In [0]:
%python
matches_df = matches_df.withColumn("HomeTeamGoals", (rand() * 6).cast("int")) 
matches_df = matches_df.withColumn("AwayTeamGoals", (rand() * 6).cast("int")) 


matches_df.createOrReplaceTempView("matches_with_goals")

In [0]:
%python
matches_df.show()

+--------------+----------------+-------------+-------------+-------------+-------------+
|      HomeTeam|        AwayTeam|HomeTeamGroup|AwayTeamGroup|HomeTeamGoals|AwayTeamGoals|
+--------------+----------------+-------------+-------------+-------------+-------------+
|      Slovakia|Playoff Winner 2|            E|            E|            4|            1|
|Czech Republic|Playoff Winner 3|            F|            F|            5|            5|
|Czech Republic|          Turkey|            F|            F|            3|            5|
|Czech Republic|        Portugal|            F|            F|            2|            3|
|      Portugal|Playoff Winner 3|            F|            F|            2|            1|
|      Portugal|          Turkey|            F|            F|            5|            1|
|        Turkey|Playoff Winner 3|            F|            F|            4|            2|
|       Albania|           Spain|            B|            B|            3|            4|
|       Al

In [0]:
%python


## Calculate additional match statistics
matches_with_stats_df = spark.sql("""
    SELECT 
        TeamName AS Team,
        TeamGroup,
        COUNT(*) AS Pld,
        SUM(CASE WHEN HomeTeamGoals = AwayTeamGoals THEN 1 ELSE 0 END) AS D,
        SUM(CASE WHEN HomeTeamGoals > AwayTeamGoals THEN 1 ELSE 0 END) AS W,
        SUM(CASE WHEN HomeTeamGoals < AwayTeamGoals THEN 1 ELSE 0 END) AS L,
        SUM(HomeTeamGoals) AS GF,
        SUM(AwayTeamGoals) AS GA,
        SUM(HomeTeamGoals) - SUM(AwayTeamGoals) AS GD,
        SUM(
            CASE 
                WHEN HomeTeamGoals > AwayTeamGoals THEN 3
                WHEN HomeTeamGoals = AwayTeamGoals THEN 1
                ELSE 0 
            END
        ) AS Pts
      
    FROM (
        SELECT HomeTeam AS TeamName, HomeTeamGoals, AwayTeamGoals, HomeTeamGroup AS TeamGroup FROM matches_with_goals
        UNION ALL
        SELECT AwayTeam AS TeamName, AwayTeamGoals, HomeTeamGoals, AwayTeamGroup AS TeamGroup FROM matches_with_goals
    )
    GROUP BY TeamName, TeamGroup
    ORDER BY Pts DESC, GD DESC
""")


In [0]:
%python
matches_df.show()

+--------------+----------------+-------------+-------------+-------------+-------------+
|      HomeTeam|        AwayTeam|HomeTeamGroup|AwayTeamGroup|HomeTeamGoals|AwayTeamGoals|
+--------------+----------------+-------------+-------------+-------------+-------------+
|      Slovakia|Playoff Winner 2|            E|            E|            4|            1|
|Czech Republic|Playoff Winner 3|            F|            F|            5|            5|
|Czech Republic|          Turkey|            F|            F|            3|            5|
|Czech Republic|        Portugal|            F|            F|            2|            3|
|      Portugal|Playoff Winner 3|            F|            F|            2|            1|
|      Portugal|          Turkey|            F|            F|            5|            1|
|        Turkey|Playoff Winner 3|            F|            F|            4|            2|
|       Albania|           Spain|            B|            B|            3|            4|
|       Al

In [0]:
%python
# Create a window specification to partition by Group and order by Points and Goal Difference
windowSpec = Window.partitionBy(matches_with_stats_df["TeamGroup"]).orderBy(matches_with_stats_df["Pts"].desc(), matches_with_stats_df["GD"].desc())

# Assign positions within each group
matches_with_stats_df = matches_with_stats_df.withColumn("Pos", row_number().over(windowSpec))

# Display matches table with statistics and positions within each group
# matches_with_stats_df.show()

In [0]:
%python 
matches_with_stats_df.createOrReplaceTempView("match_table");

In [0]:
%sql
SELECT * FROM match_table

Team,TeamGroup,Pld,D,W,L,GF,GA,GD,Pts,Pos
Germany,A,3,0,2,1,7,6,1,6,1
Switzerland,A,3,0,1,2,6,8,-2,3,2
Scotland,A,3,0,1,2,4,8,-4,3,3
Hungary,A,3,1,0,2,5,10,-5,1,4
Croatia,B,3,0,2,1,11,6,5,6,1
Albania,B,3,0,2,1,8,6,2,6,2
Italy,B,3,0,1,2,4,5,-1,3,3
Spain,B,3,0,1,2,6,11,-5,3,4
Denmark,C,3,0,3,0,11,3,8,9,1
England,C,3,0,2,1,7,4,3,6,2


# Round of 16

In [0]:
%sql
CREATE TABLE Combinations1 (
  ID INTEGER,
  GroupId VARCHAR(4),
  Team1B CHAR(1),
  Team1C CHAR(1),
  Team1E CHAR(1),
  Team1F CHAR(1)
);

In [0]:
%sql


INSERT INTO Combinations1 (ID, GroupId, Team1B, Team1C, Team1E, Team1F) VALUES
(1, 'ABCD', 'A', 'D', 'B', 'C'),
(2, 'ABCE', 'A', 'E', 'B', 'C'),
(3, 'ABCF', 'A', 'F', 'B', 'C'),
(4, 'ABDE', 'D', 'E', 'A', 'B'),
(5, 'ABDF', 'D', 'F', 'A', 'B'),
(6, 'ABEF', 'E', 'F', 'B', 'A'),
(7, 'ACDE', 'E', 'D', 'C', 'A'),
(8, 'ACDF', 'F', 'D', 'C', 'A'),
(9, 'ACEF', 'E', 'F', 'C', 'A'),
(10, 'ADEF', 'E', 'F', 'D', 'A'),
(11, 'BCDE', 'E', 'D', 'B', 'C'),
(12, 'BCDF', 'F', 'D', 'C', 'B'),
(13, 'BCEF', 'F', 'E', 'C', 'B'),
(14, 'BDEF', 'F', 'E', 'D', 'B'),
(15, 'CDEF', 'F', 'E', 'D', 'C');


-- select * from Combinations1

num_affected_rows,num_inserted_rows
15,15


In [0]:
%sql
SELECT * FROM match_table

Team,TeamGroup,Pld,D,W,L,GF,GA,GD,Pts,Pos
Germany,A,3,0,2,1,7,6,1,6,1
Switzerland,A,3,0,1,2,6,8,-2,3,2
Scotland,A,3,0,1,2,4,8,-4,3,3
Hungary,A,3,1,0,2,5,10,-5,1,4
Croatia,B,3,0,2,1,11,6,5,6,1
Albania,B,3,0,2,1,8,6,2,6,2
Italy,B,3,0,1,2,4,5,-1,3,3
Spain,B,3,0,1,2,6,11,-5,3,4
Denmark,C,3,0,3,0,11,3,8,9,1
England,C,3,0,2,1,7,4,3,6,2


In [0]:
%sql
CREATE TABLE ro16(
  MID INT,
  Team1 VARCHAR(25),
  Team2 VARCHAR(25)
);


In [0]:
%sql
WITH third_guys AS (
    SELECT 
        TeamGroup,
        ROW_NUMBER() OVER (ORDER BY Pts, GD) AS rank 
    FROM 
        match_table 
    WHERE 
        pos = 3 
    LIMIT 4
),
GrpConcat AS (
    SELECT 
         CONCAT_WS('', sort_array(COLLECT_LIST(TeamGroup))) AS grp
    FROM 
        third_guys 
),
combo AS (
    SELECT * 
    FROM combinations1 
    WHERE GroupId = (SELECT grp FROM GrpConcat)
)

INSERT INTO ro16 (
    SELECT 1 AS Mid,
        (SELECT Team FROM match_table WHERE Pos = 1 AND TeamGroup = 'B') as Team1B,
        (SELECT Team FROM match_table WHERE Pos = 3 AND TeamGroup = (SELECT Team1B FROM combo)) as Team2B
    UNION ALL
    SELECT 2 as Mid,
        (SELECT Team FROM match_table WHERE Pos = 1 AND TeamGroup = 'A') as Team1A,
        (SELECT Team FROM match_table WHERE Pos = 2 AND TeamGroup = 'C') as Team2C
    UNION ALL
    SELECT 3 AS Mid,
        (SELECT Team FROM match_table WHERE Pos = 1 AND TeamGroup = 'F') as Team1F,
        (SELECT Team FROM match_table WHERE Pos = 3 AND TeamGroup = (SELECT Team1F FROM combo)) as Team2F
    UNION ALL
    SELECT 4 as Mid,
        (SELECT Team FROM match_table WHERE Pos = 2 AND TeamGroup = 'D') as Team1D,
        (SELECT Team FROM match_table WHERE Pos = 2 AND TeamGroup = 'E') as Team2E
    UNION ALL
    SELECT 5 AS Mid,
        (SELECT Team FROM match_table WHERE Pos = 1 AND TeamGroup = 'E') as Team1E,
        (SELECT Team FROM match_table WHERE Pos = 3 AND TeamGroup = (SELECT Team1E FROM combo)) as Team2E
    UNION ALL
    SELECT 6 as Mid,
        (SELECT Team FROM match_table WHERE Pos = 1 AND TeamGroup = 'D') as Team1D,
        (SELECT Team FROM match_table WHERE Pos = 2 AND TeamGroup = 'F') as Team2F
    UNION ALL
    SELECT 7 AS Mid,
        (SELECT Team FROM match_table WHERE Pos = 1 AND TeamGroup = 'C') as Team1C,
        (SELECT Team FROM match_table WHERE Pos = 3 AND TeamGroup = (SELECT Team1C FROM combo)) as Team2C
    UNION ALL
    SELECT 8 as Mid,
        (SELECT Team FROM match_table WHERE Pos = 2 AND TeamGroup = 'A') as Team2A,
        (SELECT Team FROM match_table WHERE Pos = 2 AND TeamGroup = 'B') as Team2B
    FROM combo
);


num_affected_rows,num_inserted_rows
8,8


In [0]:
%sql
SELECT * FROM ro16 ORDER BY MID

MID,Team1,Team2
1,Croatia,Playoff Winner 1
2,Germany,England
3,Portugal,Italy
4,Netherlands,Romania
5,Playoff Winner 2,Scotland
6,France,Playoff Winner 3
7,Denmark,Turkey
8,Switzerland,Albania


In [0]:
%sql

CREATE TABLE pgs_matches (
    Match STRING,
    Team1 VARCHAR(50),
    Team2 VARCHAR(50),
    Team1Goals INT,
    Team2Goals INT,
    Winner VARCHAR(50)
);

In [0]:
%sql

INSERT INTO pgs_matches (
    SELECT
        "RS " || ROW_NUMBER() OVER (ORDER BY MID) AS Match,
        Team1 AS Team1,
        Team2 AS Team2,
        FLOOR(rand() * 5) AS Team1Goals,
        FLOOR(rand() * 5) AS Team2Goals,
        CASE
            WHEN Team1Goals > Team2Goals THEN Team1
            WHEN Team2Goals > Team1Goals THEN Team2
            ELSE CASE WHEN rand() > 0.5 THEN Team1 ELSE Team2 END
        END AS Winner
    FROM ro16
);


num_affected_rows,num_inserted_rows
8,8


In [0]:
%sql
-- Insert quarter-final matches into pgs_matches table
INSERT INTO pgs_matches (
    WITH QuarterFinal AS (
        SELECT 'QF 1' AS QuarterFinalMatch, 
               (SELECT Winner FROM pgs_matches WHERE Match = 'RS 1') AS Team1,
               (SELECT Winner FROM pgs_matches WHERE Match = 'RS 2') AS Team2
        UNION ALL
        SELECT 'QF 2' AS QuarterFinalMatch, 
               (SELECT Winner FROM pgs_matches WHERE Match = 'RS 3') AS Team1,
               (SELECT Winner FROM pgs_matches WHERE Match = 'RS 4') AS Team2
        UNION ALL
        SELECT 'QF 3' AS QuarterFinalMatch, 
               (SELECT Winner FROM pgs_matches WHERE Match = 'RS 5') AS Team1,
               (SELECT Winner FROM pgs_matches WHERE Match = 'RS 6') AS Team2
        UNION ALL
        SELECT 'QF 4' AS QuarterFinalMatch, 
               (SELECT Winner FROM pgs_matches WHERE Match = 'RS 7') AS Team1,
               (SELECT Winner FROM pgs_matches WHERE Match = 'RS 8') AS Team2
    )
    SELECT
        QuarterFinalMatch,
        Team1 AS Team1Name,
        Team2 AS Team2Name,
        ROUND(RAND() * 5) AS Team1Goals,
        ROUND(RAND() * 5) AS Team2Goals,
        CASE
            WHEN ROUND(RAND()) = 1 THEN Team1
            ELSE Team2
        END AS Winner
    FROM QuarterFinal
);


num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
INSERT INTO pgs_matches (
    WITH SemiFinal AS (
        SELECT 'SF 1' AS SemiFinalMatch, 
               (SELECT Winner FROM pgs_matches WHERE Match = 'QF 1') AS Team1,
               (SELECT Winner FROM pgs_matches WHERE Match = 'QF 2') AS Team2
        UNION ALL
        SELECT 'SF 2' AS SemiFinalMatch, 
               (SELECT Winner FROM pgs_matches WHERE Match = 'QF 3') AS Team1,
               (SELECT Winner FROM pgs_matches WHERE Match = 'QF 4') AS Team2
    )
    SELECT
        SemiFinalMatch,
        Team1 AS Team1Name,
        Team2 AS Team2Name,
        ROUND(RAND() * 5) AS Team1Goals,
        ROUND(RAND() * 5) AS Team2Goals,
        CASE
            WHEN ROUND(RAND()) = 1 THEN Team1
            ELSE Team2
        END AS Winner
    FROM SemiFinal
);


num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
-- Insert third-place match into pgs_matches table
INSERT INTO pgs_matches (
    WITH SemiFinalLosers AS (
        SELECT 'SF 1' AS SemiFinalMatch, 
               (SELECT CASE WHEN Winner = Team1 THEN Team2 ELSE Team1 END FROM pgs_matches WHERE Match = 'SF 1') AS Loser1,
               (SELECT CASE WHEN Winner = Team1 THEN Team2 ELSE Team1 END FROM pgs_matches WHERE Match = 'SF 2') AS Loser2
    ),
    ThirdPlace AS (
        SELECT
            'Third Place' AS ThirdPlaceMatch,
            Loser1 AS Team1Name,
            Loser2 AS Team2Name,
            ROUND(RAND() * 5) AS Team1Goals,
            ROUND(RAND() * 5) AS Team2Goals,
            CASE
                WHEN ROUND(RAND()) = 1 THEN Loser1
                ELSE Loser2
            END AS Winner
        FROM SemiFinalLosers
    )
    SELECT * FROM ThirdPlace
);


num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
-- Insert final match into pgs_matches table
INSERT INTO pgs_matches (
    WITH FinalMatch AS (
        SELECT 'Final' AS FinalMatch,
               (SELECT Winner FROM pgs_matches WHERE Match = 'SF 1') AS Team1,
               (SELECT Winner FROM pgs_matches WHERE Match = 'SF 2') AS Team2
    )
    SELECT
        FinalMatch,
        Team1 AS Team1Name,
        Team2 AS Team2Name,
        ROUND(RAND() * 5) AS Team1Goals,
        ROUND(RAND() * 5) AS Team2Goals,
        CASE
            WHEN ROUND(RAND()) = 1 THEN Team1
            ELSE Team2
        END AS Winner
    FROM FinalMatch
);


num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
SELECT * From pgs_matches

Match,Team1,Team2,Team1Goals,Team2Goals,Winner
RS 1,Croatia,Playoff Winner 1,2,4,Playoff Winner 1
RS 2,Germany,England,2,2,Germany
RS 3,Portugal,Italy,2,4,Italy
RS 4,Netherlands,Romania,0,0,Netherlands
RS 5,Playoff Winner 2,Scotland,3,4,Scotland
RS 6,France,Playoff Winner 3,2,1,France
RS 7,Denmark,Turkey,3,2,Denmark
RS 8,Switzerland,Albania,1,4,Albania
QF 1,Playoff Winner 1,Germany,5,2,Germany
Third Place,Germany,Albania,2,2,Albania


In [0]:
%sql
SELECT * FROM match_table

Team,TeamGroup,Pld,D,W,L,GF,GA,GD,Pts,Pos
Germany,A,3,0,2,1,7,6,1,6,1
Switzerland,A,3,0,1,2,6,8,-2,3,2
Scotland,A,3,0,1,2,4,8,-4,3,3
Hungary,A,3,1,0,2,5,10,-5,1,4
Croatia,B,3,0,2,1,11,6,5,6,1
Albania,B,3,0,2,1,8,6,2,6,2
Italy,B,3,0,1,2,4,5,-1,3,3
Spain,B,3,0,1,2,6,11,-5,3,4
Denmark,C,3,0,3,0,11,3,8,9,1
England,C,3,0,2,1,7,4,3,6,2


In [0]:
%sql
WITH MatchDetails AS (
    SELECT 
    CASE 
        WHEN Match IN ('RS 1', 'RS 5') THEN 3
        WHEN Match IN ('QF 1', 'QF 3') THEN 4
        WHEN Match IN ('RS 2', 'RS 6') THEN 5
        WHEN Match IN ('RS 3', 'RS 7') THEN 7
        WHEN Match IN ('QF 2', 'QF 4') THEN 8
        WHEN Match IN ('RS 4', 'RS 8') THEN 9
        WHEN Match = 'SF 1' THEN 6
        WHEN Match = 'SF 2' THEN 6
        WHEN Match = 'Final' THEN 6
        WHEN Match = 'Third Place' THEN 8
    END AS RowNum,
        

   

    

        CASE 
            WHEN Match = 'RS 1' THEN CONCAT('RS 1: ', Team1, ' vs ', Team2, ', ', Team1Goals, '-', Team2Goals, ', Winner: ', Winner)
            WHEN Match = 'RS 2' THEN CONCAT('RS 2: ', Team1, ' vs ', Team2, ', ', Team1Goals, '-', Team2Goals, ', Winner: ', Winner)
            WHEN Match = 'RS 3' THEN CONCAT('RS 3: ', Team1, ' vs ', Team2, ', ', Team1Goals, '-', Team2Goals, ', Winner: ', Winner)
            WHEN Match = 'RS 4' THEN CONCAT('RS 4: ', Team1, ' vs ', Team2, ', ', Team1Goals, '-', Team2Goals, ', Winner: ', Winner)
            ELSE '' 
        END AS PoolA,
        CASE 
            WHEN Match = 'QF 1' THEN CONCAT('QF 1: ', Team1, ' vs ', Team2, ', ', Team1Goals, '-', Team2Goals, ', Winner: ', Winner)
            WHEN Match = 'QF 2' THEN CONCAT('QF 2: ', Team1, ' vs ', Team2, ', ', Team1Goals, '-', Team2Goals, ', Winner: ', Winner)
            ELSE '' 
        END AS QFA,
        CASE 
            WHEN Match = 'SF 1' THEN CONCAT('SF 1: ', Team1, ' vs ', Team2, ', ', Team1Goals, '-', Team2Goals, ', Winner: ', Winner)
            ELSE '' 
        END AS Semi_Final_1,
        CASE 
            WHEN Match = 'Final' THEN CONCAT('Final: ', Team1, ' vs ', Team2, ', ', Team1Goals, '-', Team2Goals, ', Winner: ', Winner)
            WHEN Match = 'Third Place' THEN CONCAT('Third: ', Team1, ' vs ', Team2, ', ', Team1Goals, '-', Team2Goals, ', Winner: ', Winner)
            ELSE '' 
        END AS Final,
        CASE 
            WHEN Match = 'SF 2' THEN CONCAT('SF 2: ', Team1, ' vs ', Team2, ', ', Team1Goals, '-', Team2Goals, ', Winner: ', Winner)
            ELSE '' 
        END AS Semi_Final_2,
        CASE 
            WHEN Match = 'QF 3' THEN CONCAT('QF 3: ', Team1, ' vs ', Team2, ', ', Team1Goals, '-', Team2Goals, ', Winner: ', Winner)
            WHEN Match = 'QF 4' THEN CONCAT('QF 4: ', Team1, ' vs ', Team2, ', ', Team1Goals, '-', Team2Goals, ', Winner: ', Winner)
            ELSE '' 
        END AS QFB,
        CASE 
            WHEN Match = 'RS 5' THEN CONCAT('RS 5: ', Team1, ' vs ', Team2, ', ', Team1Goals, '-', Team2Goals, ', Winner: ', Winner)
            WHEN Match = 'RS 6' THEN CONCAT('RS 6: ', Team1, ' vs ', Team2, ', ', Team1Goals, '-', Team2Goals, ', Winner: ', Winner)
            WHEN Match = 'RS 7' THEN CONCAT('RS 7: ', Team1, ' vs ', Team2, ', ', Team1Goals, '-', Team2Goals, ', Winner: ', Winner)
            WHEN Match = 'RS 8' THEN CONCAT('RS 8: ', Team1, ' vs ', Team2, ', ', Team1Goals, '-', Team2Goals, ', Winner: ', Winner)
            ELSE '' 
        END AS PoolB


    FROM 
        pgs_matches
      
)

SELECT 
 

    
    PoolA,
    QFA,
    Semi_Final_1,
    Final,
    Semi_Final_2,
    QFB,
    PoolB
FROM 
    MatchDetails
WHERE 
    RowNum IS NOT NULL
ORDER BY 
    RowNum;


PoolA,QFA,Semi_Final_1,Final,Semi_Final_2,QFB,PoolB
"RS 1: Croatia vs Playoff Winner 1, 2-4, Winner: Playoff Winner 1",,,,,,
,,,,,,"RS 5: Playoff Winner 2 vs Scotland, 3-4, Winner: Scotland"
,,,,,"QF 3: Scotland vs France, 5-1, Winner: Scotland",
,"QF 1: Playoff Winner 1 vs Germany, 5-2, Winner: Germany",,,,,
"RS 2: Germany vs England, 2-2, Winner: Germany",,,,,,
,,,,,,"RS 6: France vs Playoff Winner 3, 2-1, Winner: France"
,,,"Final: Italy vs Scotland, 5-2, Winner: Scotland",,,
,,,,"SF 2: Scotland vs Albania, 4-3, Winner: Scotland",,
,,"SF 1: Germany vs Italy, 5-1, Winner: Italy",,,,
"RS 3: Portugal vs Italy, 2-4, Winner: Italy",,,,,,
